Example usage of FlowClient to run daily location, modal location and flow queries.

In [2]:
import flowclient

## Create an access token
Generate an API access token using [FlowAuth](../../index.html#flowauth).

## Create an API connection
Paste the access token from FlowAuth here as `TOKEN`

In [3]:
conn = flowclient.connect("http://localhost:9090", TOKEN)

## Daily location

### Specify query parameters

In [4]:
query_spec = flowclient.daily_location(
    date="2016-01-01",
    aggregation_unit="admin3",
    method="last",
    subscriber_subset=None,
)

query_spec

><div class="codehilite"><pre>{'params': {'aggregation_unit': 'admin3',
            'daily_location_method': 'last',
            'date': '2016-01-01',
            'subscriber_subset': 'all'},
 'query_kind': 'daily_location'}</pre></div>

### Set query running

In [5]:
query_id = flowclient.run_query(conn, query_spec)
query_id

>`3c875ae11876e660b3cbda68612bdb76`

### Check status of query

In [6]:
flowclient.get_status(conn, query_id)

>`Finished`

### Get query result

In [7]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

pcod  total
0  524 4 11 60   1277
1  524 3 09 50     11
2  524 1 03 11    649
3  524 3 07 40    538
4  524 1 03 14   1282

Alternative (run query, wait for it to finish, and get result, in one function call):

In [8]:
df = flowclient.get_result(conn, query_spec)
df.head()

pcod  total
0  524 4 11 60   1277
1  524 3 09 50     11
2  524 1 03 11    649
3  524 3 07 40    538
4  524 1 03 14   1282

## Modal location
### Specify query parameters
Either explicitly from a set of daily locations:

In [9]:
dl1 = flowclient.daily_location(
    date="2016-01-01",
    aggregation_unit="admin3",
    method="last",
    subscriber_subset=None,
)
dl2 = flowclient.daily_location(
    date="2016-01-02",
    aggregation_unit="admin3",
    method="last",
    subscriber_subset=None,
)
ml = flowclient.modal_location(dl1, dl2, aggregation_unit="admin3")

ml

><div class="codehilite"><pre>{'params': {'aggregation_unit': 'admin3',
            'locations': ({'params': {'aggregation_unit': 'admin3',
                                      'daily_location_method': 'last',
                                      'date': '2016-01-01',
                                      'subscriber_subset': 'all'},
                           'query_kind': 'daily_location'},
                          {'params': {'aggregation_unit': 'admin3',
                                      'daily_location_method': 'last',
                                      'date': '2016-01-02',
                                      'subscriber_subset': 'all'},
                           'query_kind': 'daily_location'})},
 'query_kind': 'modal_location'}</pre></div>

Or from a date range:

In [10]:
ml = flowclient.modal_location_from_dates(
    "2016-01-01",
    "2016-01-03",
    aggregation_unit="admin3",
    method="last",
    subscriber_subset=None,
)

ml

><div class="codehilite"><pre>{'params': {'aggregation_unit': 'admin3',
            'locations': ({'params': {'aggregation_unit': 'admin3',
                                      'daily_location_method': 'last',
                                      'date': '2016-01-01',
                                      'subscriber_subset': 'all'},
                           'query_kind': 'daily_location'},
                          {'params': {'aggregation_unit': 'admin3',
                                      'daily_location_method': 'last',
                                      'date': '2016-01-02',
                                      'subscriber_subset': 'all'},
                           'query_kind': 'daily_location'},
                          {'params': {'aggregation_unit': 'admin3',
                                      'daily_location_method': 'last',
                                      'date': '2016-01-03',
                                      'subscriber_subset': 'all'},
                           'query_kind': 'daily_location'})},
 'query_kind': 'modal_location'}</pre></div>

### Set query running

In [11]:
query_id = flowclient.run_query(conn, ml)
query_id

>`3cc88943b5fd2cd356e2d71443cc6bfd`

### Check status of query

In [12]:
flowclient.get_status(conn, query_id)

>`executing`

### Get query result

In [13]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

pcod  total
0  524 4 11 60   1506
1  524 3 09 50      7
2  524 1 03 11    750
3  524 3 07 40    664
4  524 1 03 14   1506

Alternative:

In [14]:
df = flowclient.get_result(conn, ml)
df.head()

pcod  total
0  524 4 11 60   1506
1  524 3 09 50      7
2  524 1 03 11    750
3  524 3 07 40    664
4  524 1 03 14   1506

## Flow
### Specify query parameters
Specify parameters for two location queries:

In [15]:
aggregation_unit = "admin1"
dl_past = flowclient.daily_location(
    "2016-01-01",
    aggregation_unit=aggregation_unit,
    method="last",
    subscriber_subset=None,
)
dl_present = flowclient.daily_location(
    "2016-01-07",
    aggregation_unit=aggregation_unit,
    method="last",
    subscriber_subset=None,
)

In [16]:
flowclient.run_query(conn, dl_past)
flowclient.run_query(conn, dl_present)

>`fbb40425fe8b3a86f430f6e9b330721c`

Specify parameters for flow query:

In [17]:
flow = flowclient.flows(dl_past, dl_present, aggregation_unit=aggregation_unit)
flow

><div class="codehilite"><pre>{'params': {'aggregation_unit': 'admin1',
            'from_location': {'params': {'aggregation_unit': 'admin1',
                                         'daily_location_method': 'last',
                                         'date': '2016-01-01',
                                         'subscriber_subset': 'all'},
                              'query_kind': 'daily_location'},
            'to_location': {'params': {'aggregation_unit': 'admin1',
                                       'daily_location_method': 'last',
                                       'date': '2016-01-07',
                                       'subscriber_subset': 'all'},
                            'query_kind': 'daily_location'}},
 'query_kind': 'flows'}</pre></div>

### Set query running

In [18]:
query_id = flowclient.run_query(conn, flow)
query_id

>`93267e62aec9891645b33d078a6f3dd2`

### Check status of query

In [19]:
flowclient.get_status(conn, query_id)

>`Finished`

### Get query result

In [20]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

count pcod_from pcod_to
0    329     524 1   524 5
1    608     524 1   524 4
2    434     524 1   524 3
3    448     524 1   524 2
4   5063     524 1   524 1

## Counts of Events by Location

In [21]:
df = flowclient.get_result(conn, flowclient.location_event_counts("2016-01-01", "2016-01-08", "admin3", "hour"))
df.head()

date  hour         pcod  total
0  2016-01-01   0.0  524 1 01 01    122
1  2016-01-01   1.0  524 1 01 01    130
2  2016-01-01   2.0  524 1 01 01    128
3  2016-01-01   3.0  524 1 01 01    114
4  2016-01-01   4.0  524 1 01 01    106